<h1 align="center">Matrix Factorization - Alternating Least Squares in Python</h1>

ALS is Matrix Factorization Algorithm. Matrix Factorization decomposes a large matrix into products of matrices.<br>
<br>
R = U * V<br>
<br>
For example in recommendation systems, let us consider R as a matrix of User (Rows) and Ratings (Columns). Matrix factorization will allow us to discover the latent features that define the interactions between User and Ratings. In other words, ALS uncovers the latent features.<br>

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Exploration</a></span></li><li><span><a href="#Alternating-Least-Squares" data-toc-modified-id="Alternating-Least-Squares-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Alternating Least Squares</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li></ul></div>

## Import Libraries

In [1]:
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

FILE_PATH = "./data/Movielens/"

## Load the data
The original dataset has been proprocessed to filter out and keep only the top users and items.<br>
Please refer to the preprocessing notebook in the repo for more details.

In [2]:
## user_to_item_map={}  ## Key:= User_id, Value:= [list of items] 
## item_to_user_map={}  ## Key:= item_id, Value:=[list of users] 
## train_ratings={}      ## Key:= (User_id, item_id) Value:=Rating 
## test_ratings={}       ## Key:= (User_id, item_id) Value:=Rating 

with open(FILE_PATH+'user_to_item_map.pkl', 'rb') as fp:
    user_to_item_map=pickle.load(fp)

with open(FILE_PATH+'item_to_user_map.pkl', 'rb') as fp:
    item_to_user_map=pickle.load(fp)

with open(FILE_PATH+'train_ratings.pkl', 'rb') as fp:
    train_ratings=pickle.load(fp)

with open(FILE_PATH+'test_ratings.pkl', 'rb') as fp:
    test_ratings=pickle.load(fp)

with open(FILE_PATH+'user_statistics.pkl', 'rb') as fp:
    user_statistics=pickle.load(fp)
    
with open(FILE_PATH+'item_statistics.pkl', 'rb') as fp:
    item_statistics=pickle.load(fp)

## Data Exploration

In [3]:
n_users=len(user_to_item_map)
n_items=len(item_to_user_map)
user_ids=range(n_users)
item_ids=range(n_items)
matrix_size=n_users*n_items
n_ratings=len(train_ratings)+len(test_ratings)

print("Number of unique users:",n_users)
print("Number of unique items:",n_items)
print("Total ratings in the dataset:",n_ratings)

print("User-Item matrix size:",matrix_size)
print("User-Item matrix empty percentage:",(matrix_size-n_ratings)*100/(matrix_size))

Number of unique users: 200
Number of unique items: 600
Total ratings in the dataset: 95871
User-Item matrix size: 120000
User-Item matrix empty percentage: 20.1075


## Alternating Least Squares

In [4]:
def initialize(latent_factors):
    user_weights={}
    item_weights={}
    user_bias=defaultdict(int)
    item_bias=defaultdict(int)
    
    for user in user_ids:
        user_weights[user]=np.random.rand(latent_factors)

    for item in item_ids:
        item_weights[item]=np.random.rand(latent_factors)

    mean=np.mean(list(train_ratings.values()))
    
    return user_weights,item_weights,user_bias,item_bias,mean

In [5]:
def ALS(user_weights,item_weights,user_bias,item_bias,mean,iterations,reg,latent_factors):
    n=latent_factors
    for i in range(iterations):
        print("Iteration: {} .........".format(i))
        for user in user_ids:
            A=np.zeros([n,n])
            b=np.zeros([1,n])
            #if user not in user_to_item_map: continue
            for item in user_to_item_map[user]:
                A+=np.outer(item_weights[item],item_weights[item].T)+reg*(np.eye(n))
                b+=(train_ratings[(user,item)]-item_bias[item]-user_bias[user]-mean)*item_weights[item]

            user_weights[user]=np.linalg.solve(A,b.T).T

        for item in item_ids:
            A=np.zeros([n,n])
            b=np.zeros([1,n])
            #if item not in item_to_user_map: continue
            for user in item_to_user_map[item]:
                A+=np.outer(user_weights[user],user_weights[user].T)+reg*(np.eye(n))
                b+=(train_ratings[(user,item)]-item_bias[item]-user_bias[user]-mean)*user_weights[user]
            item_weights[item]=np.linalg.solve(A,b.T).T

        for user in user_ids:
            mlen=len(user_to_item_map[user])
            item_bias[item]=0
            for item in user_to_item_map[user]:
                item_bias[item]=(1.0/(mlen+reg))*(train_ratings[(user,item)]-np.dot(user_weights[user],item_weights[item].T)-user_bias[user]-mean)[0]

        for item in item_ids:
            ulen=len(item_to_user_map[item])
            user_bias[user]=0
            for user in item_to_user_map[item]:
                user_bias[user]=(1.0/(ulen+reg))*(train_ratings[(user,item)]-np.dot(user_weights[user],item_weights[item].T)-item_bias[item]-mean)[0]


In [6]:
def calculate_MSE(dataset,user_weights,item_weights,user_bias,item_bias,mean):
    errors=[]
    for (user,item),rating in dataset.items():
        #if user in user_weights and item in item_weights:
        pred=np.dot(user_weights[user],item_weights[item].T)[0][0]+item_bias[item]+user_bias[user]+mean
        errors.append((pred-rating)**2)

    return np.mean(errors)

## Results

In [7]:
latent_factors=10  ## No. of latent factors
reg=0.01
iterations=5
user_weights,item_weights,user_bias,item_bias,mean = initialize(latent_factors)
ALS(user_weights,item_weights,user_bias,item_bias,mean,iterations,reg,latent_factors)

print("Length of user and item weight vectors: ",latent_factors)
print("Mean rating of the train data: ",mean)

Iteration: 0 .........
Iteration: 1 .........
Iteration: 2 .........
Iteration: 3 .........
Iteration: 4 .........
Length of user and item weight vectors:  10
Mean rating of the train data:  3.495066111495438


In [8]:
print("Train error: ",calculate_MSE(train_ratings,user_weights,item_weights,user_bias,item_bias,mean))
print("Test error: ",calculate_MSE(test_ratings,user_weights,item_weights,user_bias,item_bias,mean))

Train error:  0.44707146525930824
Test error:  0.5955405833436895


## References

1. https://en.wikipedia.org/wiki/Recommender_system <br>
2. https://www.udemy.com/recommender-systems/ <br>
3. https://www.quora.com/What-is-the-Alternating-Least-Squares-method-in-recommendation-systems-And-why-does-this-algorithm-work-intuition-behind-this